In [0]:
train_test_dataset_zip = "/content/drive/My Drive/Google Colab Dataset/ucsdpeds.zip"
model_folder = f'/content/drive/My Drive/icch/model/'
model_train_test_folder = '/content/drive/My Drive/icch/train_test/coward_counting_train_test_data'

In [0]:
from google.colab import drive
import zipfile
drive.mount('/content/drive/')

In [0]:
!rm -r '/content/drive/My Drive/icch/train_test/coward_counting_train_test_data'
!mkdir '/content/drive/My Drive/icch/train_test/coward_counting_train_test_data'

In [0]:
!ls "/content/drive/My Drive/Google Colab Dataset"        

In [0]:
!unzip "/content/drive/My Drive/Google Colab Dataset/ucsdpeds.zip" -d '/content/drive/My Drive/icch/train_test/coward_counting_train_test_data'

In [0]:
!ls model_train_test_folder

In [0]:
!pip install torch
!pip install torchvision
!pip install scipy
!pip install tqdm

In [0]:
import time
import math
import torch
import torch.nn as nn
from torchvision import models
import random
import os
from PIL import Image,ImageFilter,ImageDraw
import numpy as np
import h5py
import shutil
from PIL import ImageStat
import cv2
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import os
import sys
import torch
from torch.nn import functional as TF
import numpy as np
import time
from tqdm import tqdm
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
from scipy import spatial, ndimage 
import json
from matplotlib import cm as CM
from image import *

import random

from torch.utils.data import Dataset
from torchvision import datasets, transforms
from PIL import Image
from image import *
import torchvision.transforms.functional as F
%matplotlib inline

In [0]:
def gaussian_filter_density(gt):
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048

    tree = spatial.KDTree(pts.copy(), leafsize=leafsize)

    distances, locations = tree.query(pts, k=4)

    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')

    return density

In [0]:
def image_paths(path_sets, search_extension = '*.jpg'):
  
  img_paths = []
  for path in path_sets:
      for img_path in glob.glob(os.path.join(path, search_extension)):
          img_paths.append(img_path)
  return img_paths


def generate_density(image_paths):
  for img_path in tqdm(image_paths):
      mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','gt').replace('IMG_','GT_'))
      img= plt.imread(img_path)
      k = np.zeros((img.shape[0],img.shape[1]))
      gt = mat["image_info"][0,0][0,0][0]
      for i in range(0,len(gt)):
          if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
              k[int(gt[i][1]),int(gt[i][0])]=1
      k = gaussian_filter_density(k)
      with h5py.File(img_path.replace('.jpg','.h5').replace('images','gt'), 'w') as hf:
              hf['density'] = k


In [0]:
torch.cuda.is_available()

In [0]:
root = model_train_test_folder

part_1_train = os.path.join(root,'part_1_final/train_data','images')
part_1_test = os.path.join(root,'part_1_final/test_data','images')
part_2_train = os.path.join(root,'part_2_final/train_data','images')
part_2_test = os.path.join(root,'part_2_final/test_data','images')
path_sets_A = [part_1_train,part_1_test]
path_sets_B = [part_2_train,part_2_test]


In [0]:
import multiprocessing as mp

In [0]:
image_paths_A = image_paths(path_sets_A)
image_paths_B = image_paths(path_sets_B)

In [0]:
print(image_paths_A)

In [0]:

from multiprocessing import Pool

with Pool(10) as p:
  p.map(generate_density, [image_paths_A])

with Pool(10) as p:
  p.map(generate_density, [image_paths_B])


In [0]:
best_prec1 = 1e6
  
original_lr = 1e-7
lr = 1e-7
batch_size    = 1
momentum      = 0.95
decay         = 5*1e-4
start_epoch   = 0
epochs = 400
steps         = [-1,1,100,150]
scales        = [1,1,1,1]
workers = 4
seed = time.time()
print_freq = 30
pre = None
task = 'a'

torch.cuda.manual_seed(seed)

In [0]:
def load_data(img_path,train = True):
    gt_path = img_path.replace('.jpg','.h5').replace('images','gt')
    img = Image.open(img_path).convert('RGB')
    gt_file = h5py.File(gt_path)
    target = np.asarray(gt_file['density'])
    if False:
        crop_size = (img.size[0]/2,img.size[1]/2)
        if random.randint(0,9)<= -1:
            dx = int(random.randint(0,1)*img.size[0]*1./2)
            dy = int(random.randint(0,1)*img.size[1]*1./2)
        else:
            dx = int(random.random()*img.size[0]*1./2)
            dy = int(random.random()*img.size[1]*1./2)
        
        img = img.crop((dx,dy,crop_size[0]+dx,crop_size[1]+dy))
        target = target[dy:crop_size[1]+dy,dx:crop_size[0]+dx]
        
        if random.random()>0.8:
            target = np.fliplr(target)
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
    
    target = cv2.resize(target,(target.shape[1]// 8,target.shape[0]// 8),interpolation = cv2.INTER_CUBIC)*64    
    
    return img,target

In [0]:
def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File(fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best,task_id, filename='checkpoint.pth.tar'):
    torch.save(state, task_id+filename)
    if is_best:
        shutil.copyfile(task_id+filename, task_id+'ccih.pth.tar') 

In [0]:
class CCNN(nn.Module):
    def __init__(self, load_weights=False):
        super(CCNN, self).__init__()
        self.seen = 0
        self.encode_layer = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.Conv2d(3, 64, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(3, 128, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.Conv2d(3, 128, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(3, 256, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.Conv2d(3, 256, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.Conv2d(3, 256, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(3, 512, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.Conv2d(3, 512, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True),
            nn.Conv2d(3, 512, kernel_size=3, padding=1,dilation = 1),
            conv2d, nn.ReLU(inplace=True)
        )
        self.decode_layer = nn.Sequential(
            nn.Conv2d(3, 512, kernel_size=3, padding=1,dilation = 1),
            nn.Conv2d(3, 512, kernel_size=3, padding=1,dilation = 1),
            nn.Conv2d(3, 512, kernel_size=3, padding=1,dilation = 1),
            nn.Conv2d(3, 256, kernel_size=3, padding=1,dilation = 1),
            nn.Conv2d(3, 256, kernel_size=3, padding=1,dilation = 1),
            nn.Conv2d(3, 128, kernel_size=3, padding=1,dilation = 1)
        )
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
            for i in range(len(self.frontend.state_dict().items())):
              list(self.frontend.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i][1].data[:]
      
    def forward(self,x):
        x = self.encode_layer(x)
        x = self.decode_layer(x)
        x = self.output_layer(x)
        return x
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [0]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

def train(train_list, model, criterion, optimizer, epoch):
  
    train_loader = torch.utils.data.DataLoader(
        listDataset(train_list,
                       shuffle=True,
                       transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]), 
                       train=True, 
                       seen=model.seen,
                       batch_size=batch_size,
                       num_workers=workers),
        batch_size=batch_size)
    print('epoch %d, processed %d samples, lr %.10f' % (epoch, epoch * len(train_loader.dataset), lr))
    
    model.train()
    end = time.time()
    
    for i, (img, target) in enumerate(train_loader):
        data_time.update(time.time() - end)
        img = img.cuda()
        img = Variable(img)
        
        target = target.type(torch.FloatTensor).unsqueeze(0).cuda()
        target = torch.autograd.Variable(target)
        
        optimizer.zero_grad()
        output = model(img)

        loss = criterion(output, target)
        
        loss.backward()
        clip_gradient(model, 1e-1)
        optimizer.step()    
        
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  .format(
                   epoch, i, len(train_loader), loss=loss.item()))
            
def validate(val_list, model, criterion):
    test_loader = torch.utils.data.DataLoader(
    listDataset(val_list,
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ]),  train=False),
    batch_size= batch_size)    
    
    model.eval()
    
    mae = 0
    
    for i,(img, target) in enumerate(test_loader):
        img = img.cuda()
        img = Variable(img)
        output = model(img)
        
        mae += abs(output.data.sum()-target.sum().type(torch.FloatTensor).cuda())
        
    mae = mae/len(test_loader)    
    print(' * MAE {mae:.3f} '
              .format(mae=mae))

    return mae    
        
    

In [0]:
class listDataset(Dataset):
    def __init__(self, root, shape=None, shuffle=True, transform=None,  train=False, seen=0, batch_size=1, num_workers=4):
        if train:
            root = root *4
        random.shuffle(root)
        
        self.nSamples = len(root)
        self.lines = root
        self.transform = transform
        self.train = train
        self.shape = shape
        self.seen = seen
        self.batch_size = batch_size
        self.num_workers = num_workers
        
        
    def __len__(self):
        return self.nSamples
    def __getitem__(self, index):
        assert index <= len(self), 'index range error' 
        
        img_path = self.lines[index]
        
        img,target = load_data(img_path,self.train)
        
        if self.transform is not None:
            img = self.transform(img)
        return img,target

In [0]:
train_list = image_paths([part_A_train, part_B_train])
val_list = image_paths([part_A_test, part_B_test])

In [0]:
best_prec1

In [0]:
def run_training(train_dataset, validation_dataset):
  model = CCNN()
  model = model.cuda()
  criterion = nn.MSELoss(reduction='mean')

  optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=decay)
  
  if pre:
    if os.path.isfile(pre):
      checkpoint = torch.load(pre)
      args.start_epoch = checkpoint['epoch']
      best_prec1 = checkpoint['best_prec1']
      model.load_state_dict(checkpoint['state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer'])
    else:
      print("=> no checkpoint found at '{}'".format(pre))
  
    
  for epoch in range(start_epoch, epochs):
    train(train_dataset, model, criterion, optimizer, epoch)
    prec1 = validate(validation_dataset, model, criterion)
    is_best = prec1 < best_prec1

    best_prec1 = min(prec1, best_prec1)
    print(' * best MAE {mae:.3f} '
          .format(mae=best_prec1))
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': pre,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer' : optimizer.state_dict(),
    }, is_best,task)

  save_net(f'/content/drive/My Drive/icch/model/{model_file_name}'.format(model_file_name = task+'_ccih.pth.tar'), model)


In [0]:
run_training(train_list, val_list)